[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abdu1azeez/Wasserstein-Generative-Adversarial-Network-WGAN-/blob/master/WGAN_Fashion_MNIST.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
try:
  #os.mkdir('/content/drive/My Drive/Colab Notebooks/GANS/WGAN')
  os.mkdir('/content/drive/My Drive/Colab Notebooks/GANS/WGAN/images')

except:
  ! rm -rf '/content/drive/My Drive/Colab Notebooks/GANS/WGAN/images'

os.chdir('/content/drive/My Drive/Colab Notebooks/GANS/WGAN')

In [0]:
from __future__ import print_function, division

from keras.datasets.fashion_mnist import load_data
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

In [0]:
class WGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the critic
                d_loss_real = self.critic.train_on_batch(imgs, valid)
                d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip critic weights
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [12]:
wgan = WGAN()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 32)         

In [13]:

wgan.train(epochs=3000, batch_size=32, sample_interval=50)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.999916] [G loss: 1.000158]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.999928] [G loss: 1.000167]
2 [D loss: 0.999927] [G loss: 1.000167]
3 [D loss: 0.999926] [G loss: 1.000169]
4 [D loss: 0.999925] [G loss: 1.000162]
5 [D loss: 0.999931] [G loss: 1.000165]
6 [D loss: 0.999925] [G loss: 1.000172]
7 [D loss: 0.999929] [G loss: 1.000159]
8 [D loss: 0.999932] [G loss: 1.000161]
9 [D loss: 0.999930] [G loss: 1.000166]
10 [D loss: 0.999925] [G loss: 1.000163]
11 [D loss: 0.999940] [G loss: 1.000155]
12 [D loss: 0.999930] [G loss: 1.000157]
13 [D loss: 0.999926] [G loss: 1.000155]
14 [D loss: 0.999936] [G loss: 1.000149]
15 [D loss: 0.999936] [G loss: 1.000150]
16 [D loss: 0.999938] [G loss: 1.000136]
17 [D loss: 0.999938] [G loss: 1.000135]
18 [D loss: 0.999944] [G loss: 1.000122]
19 [D loss: 0.999944] [G loss: 1.000124]
20 [D loss: 0.999947] [G loss: 1.000119]
21 [D loss: 0.999950] [G loss: 1.000114]
22 [D loss: 0.999954] [G loss: 1.000106]
23 [D loss: 0.999950] [G loss: 1.000092]
24 [D loss: 0.999947] [G loss: 1.000102]
25 [D loss: 0.999953] [G 

In [20]:
!pwd
os.chdir('/content/drive/My Drive/Colab Notebooks/GANS/WGAN/images')

/content/drive/My Drive/Colab Notebooks/GANS/WGAN


In [21]:
import os
import imageio  
  
def create_gif(filenames, duration):
  filenames = sorted(os.listdir('/content/drive/My Drive/Colab Notebooks/GANS/WGAN/images'))
  images = []
  for filename in filenames:
    print(filename)
    images.append(imageio.imread(filename))
  output_file = 'fashion_mnist.gif'
  imageio.mimsave(output_file, images, duration=duration)
  
  
if __name__ == "__main__":

  duration = 0.2 
  filenames = sorted(os.listdir('/content/drive/My Drive/Colab Notebooks/GANS/WGAN/images/'))
  
  create_gif(filenames, duration)

mnist_0.png
mnist_100.png
mnist_1000.png
mnist_1050.png
mnist_1100.png
mnist_1150.png
mnist_1200.png
mnist_1250.png
mnist_1300.png
mnist_1350.png
mnist_1400.png
mnist_1450.png
mnist_150.png
mnist_1500.png
mnist_1550.png
mnist_1600.png
mnist_1650.png
mnist_1700.png
mnist_1750.png
mnist_1800.png
mnist_1850.png
mnist_1900.png
mnist_1950.png
mnist_200.png
mnist_2000.png
mnist_2050.png
mnist_2100.png
mnist_2150.png
mnist_2200.png
mnist_2250.png
mnist_2300.png
mnist_2350.png
mnist_2400.png
mnist_2450.png
mnist_250.png
mnist_2500.png
mnist_2550.png
mnist_2600.png
mnist_2650.png
mnist_2700.png
mnist_2750.png
mnist_2800.png
mnist_2850.png
mnist_2900.png
mnist_2950.png
mnist_300.png
mnist_350.png
mnist_400.png
mnist_450.png
mnist_50.png
mnist_500.png
mnist_550.png
mnist_600.png
mnist_650.png
mnist_700.png
mnist_750.png
mnist_800.png
mnist_850.png
mnist_900.png
mnist_950.png
